In [1]:
import dlt

In [4]:
dlt.pipeline(destination='duckdb', dataset_name='mydata').run([{'id': 1, 'name': 'John'}], table_name="users")

LoadInfo(pipeline=<dlt.pipeline.pipeline.Pipeline object at 0x000002EEFDF86110>, destination_name='duckdb', destination_displayable_credentials='duckdb:///C:\\Users\\HP\\dlt_ipykernel_launcher.duckdb', staging_name=None, staging_displayable_credentials=None, destination_fingerprint='', dataset_name='mydata', loads_ids=['1699372669.694865'], load_packages=[LoadPackageInfo(load_id='1699372669.694865', package_path='C:\\Users\\HP\\.dlt\\pipelines\\dlt_ipykernel_launcher\\load\\loaded\\1699372669.694865', state='loaded', schema_name='dlt_ipykernel_launcher', schema_update={}, completed_at=DateTime(2023, 11, 7, 15, 57, 51, 607056, tzinfo=Timezone('UTC')), jobs={'started_jobs': [], 'new_jobs': [], 'completed_jobs': [LoadJobInfo(state='completed_jobs', file_path='C:\\Users\\HP\\.dlt\\pipelines\\dlt_ipykernel_launcher\\load\\loaded\\1699372669.694865\\completed_jobs\\users.ea132bee69.0.insert_values', file_size=160, created_at=DateTime(2023, 11, 7, 15, 57, 49, 850093, tzinfo=Timezone('UTC')), 

In [5]:
import dlt

pipeline = dlt.pipeline(destination="duckdb", dataset_name="country_data")

data = [
    {'country': 'USA', 'population': 331449281, 'capital': 'Washington, D.C.'},
    {'country': 'Canada', 'population': 38005238, 'capital': 'Ottawa'},
    {'country': 'Germany', 'population': 83019200, 'capital': 'Berlin'}
]

info = pipeline.run(data, table_name="countries")

print(info)

Pipeline dlt_ipykernel_launcher completed in 2.31 seconds
1 load package(s) were loaded to destination duckdb and into dataset country_data
The duckdb destination used duckdb:///C:\Users\HP\dlt_ipykernel_launcher.duckdb location to store data
Load package 1699372737.805964 is LOADED and contains no failed jobs


In [6]:
import dlt

# have data? dlt likes data
data = [{'id': 1, 'name': 'John'}]

# open connection
pipeline = dlt.pipeline(
    destination='duckdb',
    dataset_name='raw_data'
)

# Upsert/merge: Update old records, insert new
load_info = pipeline.run(
    data,
    write_disposition="merge",
    primary_key="id",
    table_name="users"
)

In [7]:
venv = dlt.dbt.get_venv(pipeline)
dbt = dlt.dbt.package(
    pipeline,
    "https://github.com/dbt-labs/jaffle_shop.git",
    venv=venv
)
models_info = dbt.run_all()

# Load metadata for monitoring and load package lineage.
# This allows for both row and column level lineage,
# as it contains schema update info linked to the loaded data
pipeline.run([load_info], table_name="loading_status", write_disposition='append')
pipeline.run([models_info], table_name="transform_status", write_disposition='append')

LoadInfo(pipeline=<dlt.pipeline.pipeline.Pipeline object at 0x000002EE80E107F0>, destination_name='duckdb', destination_displayable_credentials='duckdb:///C:\\Users\\HP\\dlt_ipykernel_launcher.duckdb', staging_name=None, staging_displayable_credentials=None, destination_fingerprint='', dataset_name='raw_data', loads_ids=['1699373568.881148'], load_packages=[LoadPackageInfo(load_id='1699373568.881148', package_path='C:\\Users\\HP\\.dlt\\pipelines\\dlt_ipykernel_launcher\\load\\loaded\\1699373568.881148', state='loaded', schema_name='dlt_ipykernel_launcher', schema_update={}, completed_at=DateTime(2023, 11, 7, 16, 12, 50, 136602, tzinfo=Timezone('UTC')), jobs={'started_jobs': [], 'new_jobs': [], 'completed_jobs': [LoadJobInfo(state='completed_jobs', file_path='C:\\Users\\HP\\.dlt\\pipelines\\dlt_ipykernel_launcher\\load\\loaded\\1699373568.881148\\completed_jobs\\transform_status.73542f32eb.0.insert_values', file_size=358, created_at=DateTime(2023, 11, 7, 16, 12, 48, 939563, tzinfo=Timez

In [8]:
from dlt.common.runtime.slack import send_slack_message

slack_hook = "https://hooks.slack.com/services/xxx/xxx/xxx"

for package in load_info.load_packages:
    for table_name, table in package.schema_update.items():
        for column_name, column in table["columns"].items():
            send_slack_message(
                slack_hook,
                message=f"\tTable updated: {table_name}: Column changed: {column_name}: {column['data_type']}"
            )

In [9]:
@dlt.resource(primary_key="id", write_disposition="merge")
def github_repo_events():
    yield from _get_event_pages()